In [29]:
!pip install pyvi

In [30]:
from random import randint
import os
import json
from pyvi import ViTokenizer
from sklearn.naive_bayes import MultinomialNB
from gensim import corpora, matutils
from sklearn.metrics import classification_report
import pandas as pd
import pickle

In [31]:
DATA_TRAIN_PATH = '/kaggle/input/vietnamese-text-classification-dskt/VNeseTextClassification/trainning.txt'
DATA_TEST_PATH = '/kaggle/input/vietnamese-text-classification-dskt/VNeseTextClassification/testing.txt'
LABEL_TEST_PATH = '/kaggle/input/vietnamese-text-classification-dskt/VNeseTextClassification/testing_labels.txt'
STOP_WORDS = '/kaggle/input/vietnamese-text-classification-dskt/stopwords-nlp-vi.txt'
SPECIAL_CHARACTER = '0123456789%@$.,=+-!;/()*"&^:#|\n\t\''
DICTIONARY_PATH = '/kaggle/working/dictionary_1.txt'

In [32]:
def convert_label(abbreviation):
    # Từ điển chuyển đổi viết tắt thành từ gốc
    label_dict = {
        '__CTXH__': 'Chính trị Xã hội',
        '__DS__': 'Đời sống',
        '__KH__': 'Khoa học',
        '__KD__': 'Kinh doanh',
        '__PL__': 'Pháp luật',
        '__SK__': 'Sức khỏe',
        '__TG__': 'Thế giới',
        '__TT__': 'Thể thao',
        '__VH__': 'Văn hóa',
        '__VT__': 'Vi tính'
    }
    return label_dict.get(abbreviation, abbreviation)

In [33]:
# Read different types of data
class FileReader(object):
    def __init__(self, file_path, encoder=None):
        self.file_path = file_path
        self.encoder = encoder if encoder is not None else 'utf-8'

    # Read stopwords from txt file
    def read_stopwords(self):
        with open(self.file_path, 'r', encoding=self.encoder) as f:
            stopwords = set([w.strip().replace(' ', '_') for w in f.readlines()])
        return stopwords

    # Read dictionary from txt file
    def load_dictionary(self):
        return corpora.Dictionary.load_from_text(self.file_path)
        

In [34]:
# Store different type of data
class FileStore(object):
    def __init__(self, file_path, data = None):
        self.file_path = file_path
        self.data = data

    def store_json(self):
        with open(self.file_path, 'w') as outfile:
            json.dump(self.data, outfile)

    # Save dictionary from set of words
    def store_dictionary(self, dict_words):
        dictionary = corpora.Dictionary(dict_words)
        dictionary.filter_extremes(no_below = 5, no_above=0.2)
        dictionary.save_as_text(self.file_path)

    def save_pickle(self,  obj):
        outfile = open(self.file_path, 'wb')
        fastPickler = pickle.Pickler(outfile, pickle.HIGHEST_PROTOCOL)
        fastPickler.fast = 1
        fastPickler.dump(obj)
        outfile.close()

In [35]:
class TrainDataLoader(object):
    def __init__(self, data_path):
        self.df = pd.read_csv(data_path, delimiter='\t', header=None, names=['label', 'content'], encoding="utf-8")
    
    def get_data(self):
        data = []
        for _, row in self.df.iterrows():
            data.append({
                    'category': convert_label(row['label']),
                    'content': row['content']
                }) 
        return data

In [36]:
class TestDataLoader:
    def __init__(self, content_path, label_path, encoding='utf-8'):
        """
        Khởi tạo đối tượng TestDataLoader.

        Args:
            content_path (str): Đường dẫn đến tệp chứa nội dung văn bản.
            label_path (str): Đường dẫn đến tệp chứa nhãn tương ứng.
            encoding (str, optional): Kiểu mã hóa của tệp (mặc định là 'utf-8').
        """
        try:
            with open(content_path, 'r', encoding=encoding) as f:
                self.content = f.readlines()
        except FileNotFoundError:
            raise FileNotFoundError(f"Không tìm thấy tệp nội dung: {content_path}")

        try:
            with open(label_path, 'r', encoding=encoding) as f:
                self.labels = f.readlines()
        except FileNotFoundError:
            raise FileNotFoundError(f"Không tìm thấy tệp nhãn: {label_path}")

        if len(self.content) != len(self.labels):
            raise ValueError("Số lượng nội dung và nhãn không khớp.")

    def get_data(self):
        """
        Trả về một danh sách các từ điển, mỗi từ điển chứa 'content' và 'label'.
        """
        data = []
        for content, label in zip(self.content, self.labels):
            data.append({
                'category': convert_label(label.strip()),
                'content': content.strip()
            })
        return data

In [37]:
class NLP(object):
    def __init__(self, text = None):
        self.text = text
        self.__set_stopwords()

    def __set_stopwords(self):
        self.stopwords = FileReader(STOP_WORDS).read_stopwords()

    def segmentation(self):
        return ViTokenizer.tokenize(self.text)

    def split_words(self):
        text = self.segmentation()
        try:
            return [x.strip(SPECIAL_CHARACTER).lower() for x in text.split()]
        except TypeError:
            return []

    def get_words_feature(self):
        split_words = self.split_words()
        return [word for word in split_words if word not in self.stopwords]

In [38]:
class FeatureExtraction(object):
    def __init__(self, data):
        self.data = data

    def build_dictionary(self):
#        print('Building dictionary')
        dict_words = []
        i = 0
        for text in self.data:
            i += 1
#             print("Step {} / {}".format(i, len(self.data)))
            words = NLP(text = text['content']).get_words_feature()
            dict_words.append(words)
        FileStore(file_path=DICTIONARY_PATH).store_dictionary(dict_words)

    def __load_dictionary(self):
        if os.path.exists(DICTIONARY_PATH) == False:
            self.build_dictionary()
        self.dictionary = FileReader(DICTIONARY_PATH).load_dictionary()

    def __build_dataset(self):
        self.features = []
        self.labels = []
        i = 0
        for d in self.data:
            i += 1
#             print("Step {} / {}".format(i, len(self.data)))
            self.features.append(self.get_dense(d['content']))
            self.labels.append(d['category'])

    def get_dense(self, text):
        self.__load_dictionary()
        words = NLP(text).get_words_feature()
        # Bag of words
        vec = self.dictionary.doc2bow(words)
        dense = list(matutils.corpus2dense([vec], num_terms=len(self.dictionary)).T[0])
        return dense

    def get_data_and_label(self):
        self.__build_dataset()
        return self.features, self.labels

In [39]:
class Classifier(object):
    def __init__(self, features_train = None, labels_train = None, features_test = None, labels_test = None,  model = MultinomialNB()):
        self.features_train = features_train
        self.features_test = features_test
        self.labels_train = labels_train
        self.labels_test = labels_test
        self.model = model

    def training(self):
        self.model.fit(self.features_train, self.labels_train)
        
    def save_model(self, file_path):
        FileStore(file_path=file_path).save_pickle(obj=self.model)

    def training_result(self):
        y_true, y_pred = self.labels_test, self.model.predict(self.features_test)
        print(classification_report(y_true, y_pred))
        
    def wrong_predictions(self, X_data):
        """
        In ra các dự đoán sai, bao gồm nội dung (content) và nhãn dự đoán/nhãn thực tế.
        X_data: List of dictionaries, mỗi dict có dạng {'label': ..., 'content': ...}
        """
        y_true, y_pred = self.labels_test, self.model.predict(self.features_test)
        
        wrong_predictions = []
        for i, (pred, true) in enumerate(zip(y_pred, y_true)):
            if pred != true:
                wrong_predictions.append({
                    'content': ' '.join(NLP(text = X_data[i]['content']).get_words_feature()),
                    'predicted_label': pred,
                    'true_label': true
                })

        if wrong_predictions:
            print("Các dự đoán sai:")
            for item in wrong_predictions:
                #print(f"- Nội dung: {item['content']}")
                print(f"  - Dự đoán: {item['predicted_label']}")
                print(f"  - Thực tế: {item['true_label']}\n")
        else:
            print("Không có dự đoán sai nào!")

In [40]:
data_train = TrainDataLoader(DATA_TRAIN_PATH).get_data()

features_train, labels_train = FeatureExtraction(data=data_train).get_data_and_label()

data_test = TestDataLoader(DATA_TEST_PATH, LABEL_TEST_PATH).get_data()

features_test, labels_test = FeatureExtraction(data=data_test).get_data_and_label()

In [41]:
naive_bayes_model = Classifier(features_train=features_train, features_test=features_test, labels_train=labels_train, labels_test=labels_test)
naive_bayes_model.training()
print('Finish processing.')

Finish processing.


In [42]:
naive_bayes_model.training_result() 

                  precision    recall  f1-score   support

Chính trị Xã hội       0.79      0.73      0.75       500
        Khoa học       0.88      0.84      0.86       500
      Kinh doanh       0.83      0.91      0.87       500
       Pháp luật       0.86      0.95      0.90       500
        Sức khỏe       0.89      0.90      0.89       500
        Thế giới       0.90      0.87      0.88       500
        Thể thao       1.00      0.95      0.97       500
         Vi tính       0.92      0.93      0.93       500
         Văn hóa       0.93      0.93      0.93       500
        Đời sống       0.86      0.87      0.86       500

        accuracy                           0.89      5000
       macro avg       0.89      0.89      0.89      5000
    weighted avg       0.89      0.89      0.89      5000



In [43]:
naive_bayes_model.wrong_predictions(data_test)

Các dự đoán sai:
  - Dự đoán: Khoa học
  - Thực tế: Đời sống

  - Dự đoán: Khoa học
  - Thực tế: Đời sống

  - Dự đoán: Khoa học
  - Thực tế: Thế giới

  - Dự đoán: Đời sống
  - Thực tế: Chính trị Xã hội

  - Dự đoán: Chính trị Xã hội
  - Thực tế: Thế giới

  - Dự đoán: Thế giới
  - Thực tế: Văn hóa

  - Dự đoán: Thế giới
  - Thực tế: Vi tính

  - Dự đoán: Khoa học
  - Thực tế: Chính trị Xã hội

  - Dự đoán: Thế giới
  - Thực tế: Văn hóa

  - Dự đoán: Văn hóa
  - Thực tế: Pháp luật

  - Dự đoán: Sức khỏe
  - Thực tế: Khoa học

  - Dự đoán: Khoa học
  - Thực tế: Vi tính

  - Dự đoán: Sức khỏe
  - Thực tế: Đời sống

  - Dự đoán: Chính trị Xã hội
  - Thực tế: Đời sống

  - Dự đoán: Pháp luật
  - Thực tế: Chính trị Xã hội

  - Dự đoán: Kinh doanh
  - Thực tế: Sức khỏe

  - Dự đoán: Pháp luật
  - Thực tế: Chính trị Xã hội

  - Dự đoán: Sức khỏe
  - Thực tế: Chính trị Xã hội

  - Dự đoán: Vi tính
  - Thực tế: Khoa học

  - Dự đoán: Chính trị Xã hội
  - Thực tế: Sức khỏe

  - Dự đoán: Kinh do

In [44]:
naive_bayes_model.save_model('naive_model.pk')